# Genres that yield highest net profit

brief description

## import the data

In [1]:
#necessary libraries and modules to import

import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [2]:
#converts tsv files

tsv_file = './zippedData/rt.movie_info.tsv.gz'
convert = pd.read_table(tsv_file, delimiter='\t')
convert.to_csv('./zippedData/rt.movie_info.csv.gz')

tsv_file_r = './zippedData/rt.reviews.tsv.gz'
convert_r = pd.read_table(tsv_file_r, delimiter='\t', encoding = 'unicode_escape')
convert_r.to_csv('./zippedData/rt.reviews.csv.gz')

In [3]:
#importing batch data using os

csv_files = glob("./zippedData/*.csv.gz")
csv_files

['./zippedData\\bom.movie_gross.csv.gz',
 './zippedData\\imdb.name.basics.csv.gz',
 './zippedData\\imdb.title.akas.csv.gz',
 './zippedData\\imdb.title.basics.csv.gz',
 './zippedData\\imdb.title.crew.csv.gz',
 './zippedData\\imdb.title.principals.csv.gz',
 './zippedData\\rt.movie_info.csv.gz',
 './zippedData\\rt.reviews.csv.gz',
 './zippedData\\tmdb.movies.csv.gz',
 './zippedData\\tn.movie_budgets.csv.gz']

In [4]:
# cleaning the filenames

csv_files_dict = {} #create a dictionary of datasets
for filename in csv_files: #create a for loop to batch clean files
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") #remove .csv file extensions
    filename_df = pd.read_csv(filename, index_col=0, encoding='utf-8') 
    csv_files_dict[filename_cleaned] = filename_df #load .csv file as dataframe using col 1 as index and encode in utf-8 and save as the cleaned filename

In [5]:
print(csv_files_dict.keys())

dict_keys(['bom_movie_gross_gz', 'imdb_name_basics_gz', 'imdb_title_akas_gz', 'imdb_title_basics_gz', 'imdb_title_crew_gz', 'imdb_title_principals_gz', 'rt_movie_info_gz', 'rt_reviews_gz', 'tmdb_movies_gz', 'tn_movie_budgets_gz'])


***Code in markdown***

#create SQLite database and tables

conn = sqlite3.connect("movies_db_f1.sqlite")
def create_sql_table_from_df(df, name, conn):
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    except Exception as e:
        print(f"could not make table {name}")
        print(e)

for name, table in datafiles_dict.items():
    create_sql_table_from_df(table, name, conn)        

# Looking at Top Grossing Films by Genre

## Initial look at the dataset

In [6]:
movie_budgets_df = csv_files_dict['tn_movie_budgets_gz'] #set new dataset variable

In [7]:
type(movie_budgets_df) #call type of variable

pandas.core.frame.DataFrame

In [8]:
movie_budgets_df.shape #return the dimensions of the dataframe array

(5782, 5)

In [9]:
movie_budgets_df.head(10) #return the top 10 rows

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"


In [10]:
movie_budgets_df[movie_budgets_df['movie'] == 'Avatar'] #return the row containing 'Avatar' in col title 'movie'

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [11]:
movie_budgets_df.info() #return basic summary of dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: object(5)
memory usage: 271.0+ KB


In [12]:
movie_budgets_df.isna().sum() #sum of all the NaN values in each col

release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

The columns of production_budget, domestic_gross and worldwide_gross seem to contain numbers but their datatype is object. Also at first glance there don't seem to be any missing values

## Data Cleaning

In [13]:
#define function called convert_amt_to_int
#cast datatype of col as string [added to avoid error], remove $, and change datatype to int64

def convert_amt_to_int(df, col):
    df[col] = df[col].astype('str').str.replace("$", "").str.replace(",", "").astype('int64') 
    return df

In [14]:
#make a list of cols to batch-change datatype with a for loop

money_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in money_cols: 
    movie_budgets_df = convert_amt_to_int(movie_budgets_df, col)

In [15]:
movie_budgets_df.info() #return basic summary of dataframe to confirm datatype changes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: int64(3), object(2)
memory usage: 271.0+ KB


In [16]:
#re-confirm no NaN values

movie_budgets_df.isna().sum()

release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

## Exploratory Data Analysis

### Checking the value counts

In [17]:
#looking at the most occuring values to see if there are any weird values
#using a for loop, returning normalized values (values as a percentage of the total values) in the first 5 rows  

for col in movie_budgets_df:
    print(f'Viewing values in col: {col}')
    print(f'Top 5 values:\n{movie_budgets_df[col].value_counts(normalize = True)[:5]}')
    print("-------------------")

Viewing values in col: release_date
Top 5 values:
Dec 31, 2014    0.004151
Dec 31, 2015    0.003978
Dec 31, 2010    0.002594
Dec 31, 2008    0.002421
Dec 31, 2009    0.002248
Name: release_date, dtype: float64
-------------------
Viewing values in col: movie
Top 5 values:
Home            0.000519
Halloween       0.000519
King Kong       0.000519
Side Effects    0.000346
The Alamo       0.000346
Name: movie, dtype: float64
-------------------
Viewing values in col: production_budget
Top 5 values:
20000000    0.039952
10000000    0.036666
30000000    0.030612
15000000    0.029920
25000000    0.029575
Name: production_budget, dtype: float64
-------------------
Viewing values in col: domestic_gross
Top 5 values:
0           0.094777
8000000     0.001557
2000000     0.001211
7000000     0.001211
10000000    0.001038
Name: domestic_gross, dtype: float64
-------------------
Viewing values in col: worldwide_gross
Top 5 values:
0          0.063473
8000000    0.001557
7000000    0.001038
2000000

Now that we know our dataframe has clean data, we can start working on the logic needed to get the top 5 grossing movies. As we have the production budget available, we can create a new column which will have the profits the movie has made.

### Looking at Top Grossing Films by Genre

#### Create a new column looking at Worldwide Gross vs. Production Budget

In [18]:
#add a new column 'worldwide_budget_gross_diff' by subtracting 'production_budget' from 'worldwide_gross' cols 

movie_budgets_df['worldwide_budget_gross_diff'] = movie_budgets_df['worldwide_gross'] - movie_budgets_df['production_budget']

movie_budgets_df.head() #call first 5 rows

,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_budget_gross_diff
id,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747


In [19]:
#### Create a new column looking at Domestic Gross vs. Production Budget

In [21]:
#add a new column 'domestic_budget_gross_diff' by subtracting 'production_budget' from 'domestic_gross' cols 

movie_budgets_df['domestic_budget_gross_diff'] = movie_budgets_df['domestic_gross'] - movie_budgets_df['production_budget']

movie_budgets_df.head(10) #call first 10 rows

,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_budget_gross_diff,domestic_budget_gross_diff
id,,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,335507625
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,-169536125
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,-307237650
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,128405868
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,303181382
6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220,630662225
7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,1748134200,378815482
8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,663420425,9420425
9,"Nov 17, 2017",Justice League,300000000,229024295,655945209,355945209,-70975705


### Adding Genre columns dataset

In [22]:
#set new dataframe from the imdb title basics dataset
imdb_title_basics_df = csv_files_dict['imdb_title_basics_gz']

In [23]:
imdb_title_basics_df.head() #call first 5 rows

,primary_title,original_title,start_year,runtime_minutes,genres
tconst,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [24]:
#using .merge and a left join since there are different col names, and setting left_on and right_on 

movie_budget_genres_df = pd.merge(movie_budgets_df, imdb_title_basics_df, left_on=  ['movie'],
                   right_on= ['primary_title'], 
                   how = 'left')

In [25]:
movie_budget_genres_df.shape #return dimensions of new array

(7221, 12)

In [35]:
movie_budget_genres_df.head(50)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_budget_gross_diff,domestic_budget_gross_diff,primary_title,original_title,start_year,runtime_minutes,genres
0,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,335507625,Avatar,Abatâ,2011.0,93.0,Horror
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,-169536125,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy"
2,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,-307237650,Dark Phoenix,Dark Phoenix,2019.0,113.0,"Action,Adventure,Sci-Fi"
3,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,128405868,Avengers: Age of Ultron,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,303181382,NaN,NaN,NaN,NaN,NaN
5,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220,630662225,NaN,NaN,NaN,NaN,NaN
6,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,1748134200,378815482,Avengers: Infinity War,Avengers: Infinity War,2018.0,149.0,"Action,Adventure,Sci-Fi"
7,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,663420425,9420425,NaN,NaN,NaN,NaN,NaN
8,"Nov 17, 2017",Justice League,300000000,229024295,655945209,355945209,-70975705,Justice League,Justice League,2017.0,120.0,"Action,Adventure,Fantasy"
9,"Nov 6, 2015",Spectre,300000000,200074175,879620923,579620923,-99925825,Spectre,Spectre,2015.0,148.0,"Action,Adventure,Thriller"


In [28]:
movie_budget_genres_df.info() # return basic summary of new dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7221 entries, 0 to 7220
Data columns (total 12 columns):
release_date                   7221 non-null object
movie                          7221 non-null object
production_budget              7221 non-null int64
domestic_gross                 7221 non-null int64
worldwide_gross                7221 non-null int64
worldwide_budget_gross_diff    7221 non-null int64
domestic_budget_gross_diff     7221 non-null int64
primary_title                  3815 non-null object
original_title                 3814 non-null object
start_year                     3815 non-null float64
runtime_minutes                3328 non-null float64
genres                         3743 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 733.4+ KB


In [34]:
movie_budget_genres_df.notna()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_budget_gross_diff,domestic_budget_gross_diff,primary_title,original_title,start_year,runtime_minutes,genres
0,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7216,True,True,True,True,True,True,True,True,True,True,True,True
7217,True,True,True,True,True,True,True,False,False,False,False,False
7218,True,True,True,True,True,True,True,False,False,False,False,False
7219,True,True,True,True,True,True,True,True,True,True,True,True


## By Genre, which films yield highest net profit?

short description

### Genre performance by year

    * Review net loss
    * Review net gain

### Genre performance by runtime

    * Review net loss
    * Review net gain

## Data Cleaning

### Dealing with datatypes

### Checking the value counts 

### Result: By Genre, which films yield the highest net profit?

## Data Visualization